In [73]:
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict, concatenate_datasets
import numpy as np
import pandas as pd

import json

In [74]:
# label: true = 0, fake = 1

In [75]:
fake_train_dataset = load_from_disk("fake_true_dataset")

In [76]:
# save to json
#fake_train_dataset["train"].to_json("fake_true_dataset.json", force_ascii=True)

# load to pandas
df = pd.DataFrame(fake_train_dataset['train'])

# transform text to list by splitting on \n
df["text"] = df["text"].apply(lambda x: x.split("\n"))

# dump to json
df.to_json("fake_true_dataset.json", force_ascii=False, indent=4)



In [77]:
fake_train_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 1
    })
})

In [78]:
fake_train_dataset['train'][0]

{'text': "Context:  \n What causes an airplane wing to fly? \n Answer: As Bernoulli's principal explains, because of the shape, air moving across the curved upper surface on top of the wing will travel faster and thus produce less pressure than the air moving more slowly along the flatter bottom of the wing's surface. This imbalance of high and low pressures results in a force called lift which is applied in the direction of the lower pressure.",
 'label': 0}

In [79]:
print(fake_train_dataset['train'][0]['text'])

Context:  
 What causes an airplane wing to fly? 
 Answer: As Bernoulli's principal explains, because of the shape, air moving across the curved upper surface on top of the wing will travel faster and thus produce less pressure than the air moving more slowly along the flatter bottom of the wing's surface. This imbalance of high and low pressures results in a force called lift which is applied in the direction of the lower pressure.


In [82]:
fake_train_dataset['train'][2]

{'text': 'Context:  \n What are the best ways to decrease sound resonance in a room? \n Answer: One effective way to decrease sound resonance in a room is to choose a room with low noise levels. You can also use acoustic treatment methods such as acoustic地板, acoustic wall materials, and acoustic speakers to block out external sound. It is also important to keep the room temperature under 75°F (24°C) to minimize the impact of external noise on sound resonance. Additionally, using a fan or air conditioning system can help to circulate fresh air and keep the room cool.',
 'label': 1}

In [83]:
print(fake_train_dataset['train'][2]["text"])

Context:  
 What are the best ways to decrease sound resonance in a room? 
 Answer: One effective way to decrease sound resonance in a room is to choose a room with low noise levels. You can also use acoustic treatment methods such as acoustic地板, acoustic wall materials, and acoustic speakers to block out external sound. It is also important to keep the room temperature under 75°F (24°C) to minimize the impact of external noise on sound resonance. Additionally, using a fan or air conditioning system can help to circulate fresh air and keep the room cool.


In [90]:
# transform dataset to pandas dataframe
fake_train_df = pd.DataFrame(fake_train_dataset['train'])
fake_train_df.head()

,text,label
0,Context: \n Write a funny and whimsical horos...,0
1,Context: \n What should I do on a free aftern...,0
2,Context: \n Could a nuclear explosion change ...,0
3,"Context: \n In music theory, what is the circ...",0
4,Context: \n What are the five best places to ...,1


In [91]:
print("Average size of text with label 0 (=True text):", np.mean(fake_train_df[fake_train_df['label'] == 0]['text'].apply(len)))
print("Average size of text with label 1 (=Fake text):", np.mean(fake_train_df[fake_train_df['label'] == 1]['text'].apply(len)))

Average size of text with label 0 (=True text): 533.0
Average size of text with label 1 (=Fake text): 197.75


In [92]:
print("Average number of newlines in text with label 0 (=True text):", np.mean(fake_train_df[fake_train_df['label'] == 0]['text'].apply(lambda x: x.count("\n"))))
print("Average number of newlines in text with label 1 (=Fake text):", np.mean(fake_train_df[fake_train_df['label'] == 1]['text'].apply(lambda x: x.count("\n"))))

Average number of newlines in text with label 0 (=True text): 2.0
Average number of newlines in text with label 1 (=Fake text): 2.0


In [26]:
str_beg = "system"
str_end = "Context:"


test_str = "question why is that system You are a helpful assistant Context: OK good"


# only print the string without everything between the first occurence of str_beg and the first occurence of str_end
def remove_string_between(test_str, str_beg, str_end):
    return test_str.split(str_beg)[0] + test_str.split(str_end)[1].strip()

print(remove_string_between(test_str, str_beg, str_end))

question why is that OK good
